# Введение в стохастическую оптимизацию

## Исходные данные

In [56]:
peoples = [('Seymour','BOS'),
            ('Franny','DAL'),
            ('Zooey','CAK'),
            ('Walt','MIA'),
            ('Buddy','ORD'),
            ('Les','OMA')]

destination = 'LGA'

flights = {
   ('LGA', 'CAK'): [('6:58', '9:01', 238), ('8:19', '11:16', 122), ('9:58', '12:56', 249), ('10:32', '13:16', 139), ('12:01', '13:41', 267), ('13:37', '15:33', 142), ('15:50', '18:45', 243), ('16:33', '18:15', 253), ('18:17', '21:04', 259), ('19:46', '21:45', 214)],
   ('DAL', 'LGA'): [('6:12', '10:22', 230), ('7:53', '11:37', 433), ('9:08', '12:12', 364), ('10:30', '14:57', 290), ('12:19', '15:25', 342), ('13:54', '18:02', 294), ('15:44', '18:55', 382), ('16:52', '20:48', 448), ('18:26', '21:29', 464), ('20:07', '23:27', 473)],
   ('LGA', 'BOS'): [('6:39', '8:09', 86), ('8:23', '10:28', 149), ('9:58', '11:18', 130), ('10:33', '12:03', 74), ('12:08', '14:05', 142), ('13:39', '15:30', 74), ('15:25', '16:58', 62), ('17:03', '18:03', 103), ('18:24', '20:49', 124), ('19:58', '21:23', 142)],
   ('LGA', 'MIA'): [('6:33', '9:14', 172), ('8:23', '11:07', 143), ('9:25', '12:46', 295), ('11:08', '14:38', 262), ('12:37', '15:05', 170), ('14:08', '16:09', 232), ('15:23', '18:49', 150), ('16:50', '19:26', 304), ('18:07', '21:30', 355), ('20:27', '23:42', 169)],
   ('LGA', 'OMA'): [('6:19', '8:13', 239), ('8:04', '10:59', 136), ('9:31', '11:43', 210), ('11:07', '13:24', 171), ('12:31', '14:02', 234), ('14:05', '15:47', 226), ('15:07', '17:21', 129), ('16:35', '18:56', 144), ('18:25', '20:34', 205), ('20:05', '21:44', 172)],
   ('OMA', 'LGA'): [('6:11', '8:31', 249), ('7:39', '10:24', 219), ('9:15', '12:03', 99), ('11:08', '13:07', 175), ('12:18', '14:56', 172), ('13:37', '15:08', 250), ('15:03', '16:42', 135), ('16:51', '19:09', 147), ('18:12', '20:17', 242), ('20:05', '22:06', 261)],
   ('CAK', 'LGA'): [('6:08', '8:06', 224), ('8:27', '10:45', 139), ('9:15', '12:14', 247), ('10:53', '13:36', 189), ('12:08', '14:59', 149), ('13:40', '15:38', 137), ('15:23', '17:25', 232), ('17:08', '19:08', 262), ('18:35', '20:28', 204), ('20:30', '23:11', 114)],
   ('LGA', 'DAL'): [('6:09', '9:49', 414), ('7:57', '11:15', 347), ('9:49', '13:51', 229), ('10:51', '14:16', 256), ('12:20', '16:34', 500), ('14:20', '17:32', 332), ('15:49', '20:10', 497), ('17:14', '20:59', 277), ('18:44', '22:42', 351), ('19:57', '23:15', 512)],
   ('LGA', 'ORD'): [('6:03', '8:43', 219), ('7:50', '10:08', 164), ('9:11', '10:42', 172), ('10:33', '13:11', 132), ('12:08', '14:47', 231), ('14:19', '17:09', 190), ('15:04', '17:23', 189), ('17:06', '20:00', 95), ('18:33', '20:22', 143), ('19:32', '21:25', 160)],
   ('ORD', 'LGA'): [('6:05', '8:32', 174), ('8:25', '10:34', 157), ('9:42', '11:32', 169), ('11:01', '12:39', 260), ('12:44', '14:17', 134), ('14:22', '16:32', 126), ('15:58', '18:40', 173), ('16:43', '19:00', 246), ('18:48', '21:45', 246), ('19:50', '22:24', 269)],
   ('MIA', 'LGA'): [('6:25', '9:30', 335), ('7:34', '9:40', 324), ('9:15', '12:29', 225), ('11:28', '14:40', 248), ('12:05', '15:30', 330), ('14:01', '17:24', 338), ('15:34', '18:11', 326), ('17:07', '20:04', 291), ('18:23', '21:35', 134), ('19:53', '22:21', 173)],
   ('BOS', 'LGA'): [('6:17', '8:26', 89), ('8:04', '10:11', 95), ('9:45', '11:50', 172), ('11:16', '13:29', 83), ('12:34', '15:02', 109), ('13:40', '15:37', 138), ('15:27', '17:18', 151), ('17:11', '18:30', 108), ('18:34', '19:36', 136), ('20:17', '22:22', 102)]
}

## Целевая функция

Целевая функция schedule_cost возвращает значение, показывающее, насколько данное решение неудовлетворительно. Возвращаемое значение должно быть тем больше, чем хуже решение.

In [57]:
import time

def get_minutes(t):
    x = time.strptime(t,'%H:%M')
    return x[3]*60 + x[4]

In [58]:
def schedule_cost(sol):
    
    totalprice = 0
    latestarrival = 0
    earliestdep = 24*60
    totalwait = 0
    
    for d in range(int(len(sol)/2)):
        origin = peoples[d][1]
        outbound = flights[(origin, destination)][sol[d]]
        returnf = flights[(destination, origin)][sol[d+1]]
        
        totalprice += outbound[2]
        totalprice += returnf[2]
        
        outbound_minutes = get_minutes(outbound[1])
        returnf_minuters = get_minutes(returnf[0])
        
        if latestarrival < outbound_minutes: 
            latestarrival = outbound_minutes
            
        if earliestdep > returnf_minuters: 
            earliestdep = returnf_minuters
        
        totalwait += latestarrival - outbound_minutes
        totalwait += returnf_minuters - earliestdep
        
    if latestarrival > earliestdep: 
        totalprice += 50
    
    return totalprice + totalwait  

## Случайный поиск

In [59]:
import random

def random_optimize(domain, costf, n=1000):
    best=99999999
    bestr=None
    
    for i in range(n):
        r = [random.randint(0, domain[i]) for i in range(len(domain))]
        
        cost = costf(r)
        if cost < best:
            best = cost
            bestr = r
        return r, best        

In [60]:
domain = []
for people in peoples:
    domain.append(len(flights[(people[1], destination)]) - 1)
    domain.append(len(flights[(destination, people[1])]) - 1)
print(domain)

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


In [61]:
for i in range(10):
    result, score = random_optimize(domain, schedule_cost)
    print (result, score)

[7, 6, 1, 6, 4, 8, 3, 4, 0, 7, 2, 1] 4908
[9, 6, 9, 0, 8, 5, 9, 9, 6, 9, 0, 3] 6774
[6, 2, 9, 0, 9, 0, 4, 4, 5, 5, 7, 8] 6884
[1, 5, 4, 5, 3, 8, 4, 9, 8, 0, 1, 1] 3932
[4, 5, 5, 3, 9, 2, 9, 5, 4, 1, 5, 3] 4521
[9, 3, 0, 5, 0, 4, 7, 8, 8, 3, 0, 2] 6859
[2, 1, 1, 1, 5, 7, 7, 3, 9, 2, 2, 0] 4148
[3, 9, 2, 0, 5, 3, 2, 3, 3, 2, 5, 1] 6015
[1, 9, 1, 1, 8, 6, 2, 7, 5, 6, 4, 3] 6076
[6, 6, 6, 8, 1, 3, 5, 5, 3, 6, 8, 3] 4527


<p>Проведя 10 запусков алгоритма случайного поиска, наименьшее значение функции издержек было достигунта при следующих аргументах: [8, 0, 7, 1, 6, 7, 0, 3, 9, 8, 3, 9]</p>

## Алгоритм спуска с горы

In [62]:
def hill_climb(domain, costf):
    sol = [random.randint(0, domain[i]) for i in range(len(domain))]
    best = costf(sol)
    
    is_stop = False
    
    while not is_stop:
        neighbors = []
        for j in range(len(domain)):
            
            if 0 < sol[j] < 9:
                neighbors.append(sol[0:j] + [sol[j]+1] + sol[j+1:])
                neighbors.append(sol[0:j] + [sol[j]-1] + sol[j+1:])

            if 0 == sol[j]:
                neighbors.append(sol[0:j] + [sol[j]+1] + sol[j+1:])

            if sol[j] == domain[j]:
                neighbors.append(sol[0:j] + [sol[j]-1] + sol[j+1:])
                
        is_stop = True
        for j in range(len(neighbors)):
            cost = costf(neighbors[j])
            if cost < best:
                is_stop = False
                best = cost
                sol = neighbors[j]

    return sol, best

In [63]:
for i in range(10):
    result, score = hill_climb(domain, schedule_cost)
    print(result, score)

[3, 3, 3, 3, 4, 3, 6, 1, 1, 5, 3, 9] 2736
[3, 5, 5, 5, 6, 7, 6, 3, 3, 4, 2, 0] 2772
[3, 6, 7, 3, 6, 3, 3, 1, 3, 5, 7, 5] 3397
[3, 3, 3, 3, 4, 6, 1, 4, 6, 1, 9, 3] 2646
[7, 6, 7, 6, 6, 7, 6, 1, 0, 2, 1, 7] 2726
[7, 6, 7, 6, 6, 7, 6, 1, 4, 5, 7, 9] 2726
[7, 6, 7, 6, 6, 7, 6, 5, 6, 2, 3, 0] 2726
[3, 5, 5, 5, 1, 2, 1, 9, 8, 4, 9, 9] 3235
[3, 3, 3, 3, 1, 2, 6, 0, 8, 7, 0, 9] 3034
[3, 5, 5, 5, 6, 7, 6, 4, 3, 1, 9, 6] 2772


<p>После 10 запусков данного алгоритма спуска, можно заметить, что значения функции издержек находятся довольно близко друг к другу, но не обязательно равны. Это связано с начальной случайной инициализацией исходного решения, от которого впоследствии алгоритм делает шаги.</p>

## Алгоритм имитации отжига

In [64]:
import math

def annealing_optimize(domain, costf, T=10000.0, cool=0.99, step=1):
    
    vec = [random.randint(0, domain[i]) for i in range(len(domain))]

    while T > 0.1:
            
            i = random.randint(0, len(domain)-1)
            
            ddir = random.randint(-step, step)

            vecb = vec[:]
            vecb[i] += ddir
            if vecb[i] < 0: 
                vecb[i] = 0
                
            elif vecb[i] > domain[i]: 
                vecb[i] = domain[i]

            ea = costf(vec)
            eb = costf(vecb)
            p = pow(math.e, (-eb-ea)/T)

            if (eb < ea or random.random() < p):
                vec = vecb      

            T = T * cool
    return vec, eb

In [65]:
for i in range(10):
    result, score = annealing_optimize(domain, schedule_cost)
    print(result, score)

[3, 3, 3, 3, 4, 3, 1, 6, 6, 1, 4, 9] 2468
[3, 6, 7, 6, 6, 4, 3, 4, 6, 5, 5, 8] 3053
[0, 3, 3, 3, 4, 3, 1, 1, 9, 7, 4, 9] 2474
[0, 3, 3, 3, 4, 3, 6, 2, 3, 3, 0, 2] 2742
[0, 3, 3, 3, 4, 3, 6, 1, 9, 7, 9, 9] 2942
[3, 5, 5, 5, 1, 2, 1, 0, 3, 3, 0, 3] 3235
[3, 5, 5, 5, 6, 7, 6, 3, 8, 1, 6, 3] 2822
[3, 3, 3, 3, 4, 3, 6, 0, 8, 4, 9, 9] 2736
[3, 3, 3, 3, 4, 3, 1, 5, 0, 5, 3, 8] 2468
[3, 6, 7, 6, 6, 7, 6, 9, 1, 0, 5, 9] 2701


В результате работы алгоритма мы получили значения функции близких дргу к другу.

## Генетические алгоритмы

In [66]:
def genetic_optimize(domain, costf, pop_size=50, step=1, mutprob=0.2, elite=0.2, maxiter=100):
    
    def mutate(vec):
        i = random.randint(0, len(domain)-1)
        if vec[i] < domain[i]:
            return vec[0:i]+[vec[i]+step]+vec[i+1:]
        else:
            return vec[0:i]+[vec[i]-step]+vec[i+1:]
        
    def crossover(r1, r2):
        i = random.randint(1, len(domain)-2)
        return r1[0:i]+r2[i:]
    
    pop = []
    for i in range(pop_size):
        vec = [random.randint(0, domain[i]) for i in range(len(domain))]
        pop.append(vec)

    topelite = int(elite*pop_size)

    for i in range(maxiter):
        scores = [(costf(v), v) for v in pop]
        scores.sort()
        ranked = [v for (s,v) in scores]

    pop = ranked[0:topelite]

    while len(pop) < pop_size:
        
        if random.random() < mutprob:
            c = random.randint(0, topelite)
            pop.append(mutate(ranked[c]))

    else:
        c1 = random.randint(0, topelite)
        c2 = random.randint(0, topelite)
        pop.append(crossover(ranked[c1], ranked[c2]))

    return scores[0][1], scores[0][0]

In [67]:
for i in range(10):
    result, score = genetic_optimize(domain, schedule_cost)
    print(result, score)

[5, 5, 4, 5, 4, 8, 6, 8, 4, 6, 6, 5] 3583
[2, 4, 3, 3, 7, 6, 0, 8, 8, 3, 0, 1] 3593
[6, 6, 7, 7, 4, 3, 0, 9, 6, 6, 9, 6] 3519
[3, 6, 6, 5, 6, 6, 0, 2, 8, 9, 5, 7] 3218
[8, 7, 7, 5, 6, 4, 6, 8, 2, 9, 3, 4] 3691
[3, 6, 4, 5, 5, 7, 2, 3, 4, 7, 5, 5] 3457
[4, 3, 4, 3, 3, 1, 3, 6, 9, 6, 9, 0] 3426
[1, 6, 6, 5, 5, 4, 4, 8, 9, 2, 8, 7] 3434
[3, 1, 2, 3, 4, 4, 2, 6, 4, 6, 0, 8] 3467
[1, 3, 4, 2, 6, 6, 6, 7, 3, 6, 1, 4] 3723


Несмотря на то, что генетический алгорим вычислительно сложнее чем остальные, он значительно хуже минимизирует функцию издержек. Таким образом, минимум функции издержек достигается при использовании алгоритма имитации отжига.
Наименьший диапазон значений функции был достигнут при использовании алгоритма спуска с горы.

## Работа с допонительными данными

Добавим некоторые данные в наш словарь рейсов и в список людей.

In [68]:
if ('Mariya', 'CDG') not in peoples: peoples.append(('Mariya', 'CDG'))
if ('Felyk', 'CDG') not in peoples: peoples.append(('Felyk', 'CDG'))

В связи с тем, что функция издержек принимает ключ-значение определенного значения (10 рейсов в каждую из сторон), то при добавлении новых записий словарь это необходимо учитывать. Если длина списка(значения) будет отлична от 10, алгоритм выйдет за пределы индекса массива.

In [69]:
flights[('CDG', 'LGA')] = [('4:58', '9:01', 500), ('9:41', '13:05', 330), ('9:58', '12:56', 249), ('10:32', '13:16', 139), ('12:01', '13:41', 267), ('13:37', '15:33', 142), ('15:50', '18:45', 243), ('16:33', '18:15', 253), ('18:17', '21:04', 259), ('19:46', '21:45', 214)]
flights[('LGA', 'CDG')] = [('0:17', '4:43', 321), ('5:23', '9:43', 432), ('5:32', '9:59', 432), ('7:23', '12:43', 432),
                           ('9:50', '12:43', 295), ('13:00', '16:31', 351), 
                        ('17:17', '20:43', 311), ('19:23', '22:02', 499), ('20:05', '23:49', 312), ('21:00', '23:59', 489)]

In [70]:
domain = []
for people in peoples:
    domain.append(len(flights[(people[1], destination)]) - 1)
    domain.append(len(flights[(destination, people[1])]) - 1)
print(domain)

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


In [71]:
print(len(flights[('CDG', 'LGA')]), len(flights[('LGA', 'CDG')]))

10 10


### Случайный поиск

In [72]:
for i in range(10):
    result, score = random_optimize(domain, schedule_cost)
    print(result, score)

[8, 3, 1, 8, 0, 3, 1, 1, 2, 1, 4, 2, 9, 6, 9, 5] 7823
[9, 3, 2, 4, 7, 6, 0, 6, 5, 1, 8, 2, 3, 8, 8, 1] 9059
[8, 6, 2, 3, 7, 5, 6, 2, 8, 0, 7, 6, 6, 2, 3, 3] 7307
[2, 9, 4, 3, 8, 9, 8, 5, 4, 1, 9, 5, 7, 6, 4, 1] 7598
[9, 6, 3, 6, 1, 6, 6, 2, 2, 8, 6, 3, 3, 9, 2, 3] 7638
[9, 1, 6, 8, 4, 1, 2, 0, 2, 2, 7, 3, 4, 1, 2, 5] 9524
[8, 6, 5, 1, 9, 2, 5, 9, 7, 0, 3, 3, 8, 6, 7, 1] 8321
[4, 6, 1, 2, 1, 7, 0, 6, 9, 4, 4, 4, 6, 0, 6, 0] 8131
[1, 1, 7, 7, 3, 4, 0, 5, 1, 7, 2, 4, 6, 6, 3, 1] 7898
[6, 8, 4, 3, 8, 3, 8, 1, 3, 7, 6, 4, 1, 5, 4, 3] 7318


### Спуск с горы

In [73]:
for i in range(10):
    result, score = hill_climb(domain, schedule_cost)
    print(result, score)

[3, 3, 3, 3, 4, 3, 5, 5, 4, 2, 3, 8, 7, 7, 2, 0] 3849
[0, 6, 7, 6, 6, 7, 6, 6, 0, 9, 2, 8, 7, 6, 5, 1] 4003
[3, 3, 3, 3, 1, 2, 5, 3, 0, 8, 0, 5, 7, 9, 5, 4] 4240
[3, 4, 5, 4, 4, 4, 3, 5, 4, 8, 2, 4, 2, 8, 4, 6] 4054
[3, 3, 3, 3, 1, 2, 1, 3, 3, 0, 6, 7, 5, 4, 0, 9] 4183
[3, 5, 5, 5, 6, 7, 6, 8, 4, 1, 0, 7, 4, 8, 5, 3] 4293
[3, 8, 8, 8, 8, 7, 9, 6, 2, 4, 4, 9, 1, 0, 0, 8] 4491
[0, 6, 7, 6, 6, 7, 6, 6, 8, 6, 8, 1, 3, 7, 2, 3] 4292
[3, 3, 3, 3, 4, 3, 3, 8, 0, 0, 5, 1, 6, 7, 0, 4] 4243
[3, 3, 3, 3, 4, 3, 3, 8, 4, 2, 8, 6, 2, 6, 6, 7] 4217


### Алгоритм имитации отжига

In [74]:
for i in range(10):
    result, score = annealing_optimize(domain, schedule_cost)
    print(result, score)

[0, 2, 2, 2, 2, 6, 6, 8, 0, 6, 1, 9, 4, 3, 7, 8] 5022
[3, 5, 5, 5, 6, 7, 1, 3, 0, 8, 8, 3, 4, 3, 7, 0] 4628
[7, 6, 7, 3, 1, 2, 1, 3, 8, 9, 6, 3, 8, 0, 9, 7] 6289
[3, 3, 3, 3, 4, 3, 5, 5, 0, 7, 9, 2, 9, 3, 2, 8] 4193
[0, 2, 2, 2, 2, 2, 3, 4, 4, 6, 3, 7, 5, 8, 7, 9] 3771
[3, 4, 5, 4, 4, 4, 5, 5, 8, 6, 9, 3, 5, 6, 8, 3] 4546
[3, 3, 3, 3, 4, 3, 3, 4, 0, 4, 6, 8, 3, 4, 7, 8] 3819
[3, 3, 3, 3, 4, 3, 5, 5, 2, 9, 2, 7, 4, 6, 4, 3] 3986
[3, 5, 5, 5, 6, 7, 6, 5, 0, 8, 3, 5, 7, 9, 1, 5] 4069
[3, 3, 3, 3, 4, 3, 5, 5, 8, 7, 2, 8, 3, 2, 2, 9] 4439


### Генетический алгоритм

In [75]:
for i in range(10):
    result, score = genetic_optimize(domain, schedule_cost)
    print(result, score)

[6, 7, 9, 8, 8, 7, 2, 6, 5, 2, 7, 9, 8, 4, 0, 8] 6273
[7, 6, 6, 4, 6, 8, 1, 2, 3, 9, 9, 4, 8, 9, 7, 2] 6127
[5, 5, 4, 5, 4, 6, 6, 5, 5, 8, 0, 9, 9, 9, 5, 0] 4752
[3, 4, 5, 4, 6, 0, 6, 6, 0, 5, 5, 5, 6, 2, 7, 5] 5890
[0, 0, 3, 0, 3, 2, 3, 1, 2, 0, 3, 5, 7, 4, 0, 7] 5186
[4, 5, 4, 3, 3, 1, 1, 1, 2, 5, 3, 4, 9, 7, 8, 3] 5947
[1, 4, 3, 5, 1, 1, 6, 8, 0, 1, 1, 3, 4, 0, 1, 1] 5778
[3, 6, 3, 5, 6, 4, 6, 3, 3, 5, 6, 9, 5, 8, 4, 8] 5387
[1, 5, 7, 7, 6, 6, 5, 8, 8, 1, 3, 7, 3, 5, 2, 3] 5427
[2, 2, 2, 2, 4, 2, 3, 5, 8, 4, 2, 4, 3, 1, 7, 1] 4727


В данном случае, алгоритм спуска с горы снова продемонстровал наименьший разброс значений функции издержек.